# Reconnaissance de chiffres manuscrits : MNIST

## Librairies et fonctions utiles

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split

from IPython.core.display import HTML # permet d'afficher du code html dans jupyter

Fonction pour tracer les courbes d'apprentissage sur l'ensemble d'apprentissage et l'ensemble de validation :

In [ ]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(est, X_train, y_train) :
    train_sizes, train_scores, test_scores = learning_curve(estimator=est, X=X_train, y=y_train, train_sizes=np.linspace(0.1, 1.0, 10),
                                                        cv=5,
                                                        n_jobs=-1)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plt.figure(figsize=(8,10))
    plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='training accuracy')
    plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
    plt.plot(train_sizes, test_mean,color='green', linestyle='--',marker='s', markersize=5,label='validation accuracy')
    plt.fill_between(train_sizes,test_mean + test_std,test_mean - test_std,alpha=0.15, color='green')
    plt.grid(b='on')
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.6, 1.0])
    plt.show()

Fonction pour tracer la courbe ROC :

In [ ]:
def plot_roc_curve(est,X_test,y_test) :
    probas = est.predict_proba(X_test)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.figure(figsize=(8,8))
    plt.title('Receiver Operating Characteristic')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
    plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
    plt.xlim([-0.05,1.2])
    plt.ylim([-0.05,1.2])
    plt.ylabel('Taux de vrais positifs')
    plt.xlabel('Taux de faux positifs')
    plt.show

## Le dataset de chiffres manuscrits MNIST

On charge le dataset MNIST :

In [ ]:
df = pd.read_csv("../input/mnist-in-csv/mnist_test.csv")

In [ ]:
df.shape

On a 785 colonnes :
* une colonne 'label' identifiant le chiffre  
* et 784 colonnes de pixels (image de 28x28 pixels "aplatie")

In [ ]:
df.head(10)

On crée la cible y (colonne 'label') :

In [ ]:
y = df['label']

et les caractéristiques X :

In [ ]:
X = df.drop(['label'], axis=1)

On sépare les ensembles d'apprentissage et de test :

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

On peut maintenant appliquer les méthodes de machine learning, mais auparavant on va visualiser les images

## Visualisation des images MNIST

Pour visualiser les images, on va convertir une ligne de 784 pixels en une matrice 28x28  
Il faut en premier transformer le dataframe X en un tableau :

In [ ]:
X1 = np.array(X)

On affiche la première ligne :

In [ ]:
print(X1[0])

On applique la méthode **reshape** pour convertir cette ligne de 784 éléments en une matrice 28x28 :

In [ ]:
image = X1[0].reshape(28,28)
print(image)

On peut maintenant afficher cette matrice :

In [ ]:
plt.imshow(image)

en niveaux de gris, sans graduation des axes, et avec le label comme titre :

In [ ]:
plt.imshow(image, cmap="gray_r")
plt.axis('off')
plt.title(y[0])

On redimensionne toutes les lignes :

In [ ]:
n_samples = len(df.index)
images = X1.reshape(n_samples,28,28)

On affiche les 50 premiers :

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(y[i])

## Machine learning

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
plot_learning_curve(rf, X, y)

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

### Exercice : tester Xgboost

In [ ]:
# Sous Jupyter, si xgboost n'est pas déjà installé
!pip install xgboost

In [ ]:
#Importation de la méthode
import xgboost as XGB
#Entrainement
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
#Prédiction
y_xgb = xgb.predict(X_test)
#Calcul du score
rf_score = accuracy_score(y_test, y_xgb)
print(rf_score)
#Matrice de confusion 
cm = confusion_matrix(y_test, y_xgb)
print(cm)
#Classification report
print(classification_report(y_test, y_xgb))
